In [1]:
from __future__ import absolute_import, division
import torch
import numpy as np
import senteval
import pickle
import copy
import operator
import embedding
import CNN_Model

PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

/home/rick/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0611 05:28:35.043094 140258928281344 deprecation_wrapper.py:119] From /home/rick/anaconda3/lib/python3.6/site-packages/tensorflow_hub/native_module.py:54: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0611 05:28:35.046478 140258928281344 deprecation_wrapper.py:119] From /home/rick/anaconda3/lib/python3.6/site-packages/tensorflow_hub/__init__.py:65: The name tf.VERSION is deprecated. Please use tf.version.VERSION instead.



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


Using TensorFlow backend.
W0611 05:28:37.974234 140258928281344 deprecation_wrapper.py:119] From /home/rick/Xi/WordEmbeddingEval/SentEval-master-new/official/transformer/model/attention_layer.py:24: The name tf.layers.Layer is deprecated. Please use tf.compat.v1.layers.Layer instead.

W0611 05:28:37.989271 140258928281344 deprecation_wrapper.py:119] From /home/rick/Xi/WordEmbeddingEval/SentEval-master-new/CNN_Model.py:73: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



In [3]:
# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    global model
    
    for i,sent in enumerate(batch):
        if len(sent) == 0:
            batch[i] = ['</p>']
    if params['embed'] == 'GPT2':
        sentences = [' '.join(s) for s in batch] 
        #output, _ = embedding.getGPT2Emb(batch,True,params[MAX_LENGTH])
        output,_ = embedding.getGPT2Emb(sentences)
    elif params['embed'] == 'Glove':
        output,_ = embedding.getGloveEmb(batch,True,-1)
    elif  params['embed'] == 'BERT':
        sentences = [' '.join(s) for s in batch] 
        output,_ = embedding.getBertEmb(sentences)
    else:
        if params['embed'] == 'ELMO':
            sentences = [' '.join(s) for s in batch] 
            output, _ = embedding.getElmoEmb(sentences) 
            
    print(output.shape)
    
    # Model 
    if params['modelType'] == 'ROI':
        output = CNN_Model.POI_Pooling(output)
    elif params['modelType'] == 'Transformer':
        output = output.astype(np.float32)
        output = CNN_Model.getTransformer(params,output)
        
        print("Output from transformer: ",output.shape)
        output = CNN_Model.POI_Pooling(output)
        print("Transformer + POI_Pooling: ",output.shape)
    elif params['modelType'] == 'CNN':
        output = CNN_Model.getCNNOutput(params,output)
    elif params['modelType'] == 'DiaCNN':
        output = CNN_Model.getDiaCNNOutput(params,output)
    else:
        raise("Can't not recognoize the model type")
        model = getModel(params,output)
        output = model.predict(output)
        output = output.reshape(output.shape[0],output.shape[2])
        
    #print(output.shape)
    return output
        

In [4]:
result = [] 
seed = 10 
max_length = 60 
MAX_LENGTH = 'max_length' 

tasks = ['MRPC' ] # , 'SICKRelatedness','SICKEntailment', 'STSBenchmark'
# 'MR', 'CR','MPQA','SUBJ','TREC', 'MRPC','SICKEntailment'
result = []
seed = 10 
for task in tasks:
    print(task)
    
    params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 10}
    params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 256,
                                     'tenacity': 5, 'epoch_size': 4,}
    
    params_senteval['seed'] = seed
    params_senteval[MAX_LENGTH] = max_length
    params_senteval['embed'] = 'BERT' # 'BERT',  #,'GPT2','Glove','ELMO'
    params_senteval['embedPath'] = ''
    params_senteval['modelType'] = 'ROI' #['CNN','DiaCNN','POI','Transformer']
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    # Set up logger
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    transfer_tasks =   [task]  
    results = se.eval(transfer_tasks)
    print(results)
    result.append(results)

MRPC


/home/rick/anaconda3/lib/python3.6/site-packages/bert_serving/client/__init__.py:277: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


(128, 31, 768)
(128, 27, 768)
(128, 29, 768)
(128, 30, 768)
(128, 35, 768)
(128, 36, 768)
(128, 36, 768)
(128, 39, 768)
(128, 37, 768)
(128, 38, 768)
(128, 38, 768)
(128, 36, 768)
(128, 40, 768)
(128, 41, 768)
(128, 40, 768)
(128, 42, 768)
(128, 45, 768)
(128, 46, 768)
(128, 50, 768)
(128, 44, 768)
(128, 44, 768)
(128, 42, 768)
(128, 47, 768)
(128, 50, 768)
(128, 46, 768)
(128, 50, 768)
(128, 49, 768)
(128, 57, 768)
(128, 55, 768)
(128, 56, 768)
(128, 59, 768)
(108, 57, 768)
(128, 39, 768)
(128, 29, 768)
(128, 38, 768)
(128, 37, 768)
(128, 40, 768)
(128, 38, 768)
(128, 33, 768)
(128, 45, 768)
(128, 40, 768)
(128, 42, 768)
(128, 40, 768)
(128, 47, 768)
(128, 42, 768)
(128, 44, 768)
(128, 40, 768)
(128, 52, 768)
(128, 52, 768)
(128, 58, 768)
(128, 56, 768)
(128, 47, 768)
(128, 51, 768)
(128, 44, 768)
(128, 53, 768)
(128, 51, 768)
(128, 47, 768)
(128, 54, 768)
(128, 52, 768)
(128, 54, 768)
(128, 49, 768)
(128, 60, 768)
(128, 57, 768)
(108, 57, 768)
(128, 28, 768)
(128, 31, 768)
(128, 33, 

In [8]:
result

[{'TREC': {'devacc': 63.99, 'acc': 72.6, 'ndev': 5452, 'ntest': 500}},
 {'MRPC': {'devacc': 70.9,
   'acc': 69.04,
   'f1': 80.51,
   'ndev': 4076,
   'ntest': 1725}},
 {'SICKEntailment': {'devacc': 70.2,
   'acc': 68.74,
   'ndev': 500,
   'ntest': 4927}}]